# Hawaii - A Climate Analysis And Exploration
### For data between August 23, 2016 - August 23, 2017
---

In [2]:
# Import dependencies

%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

## Reflect Tables into SQLAlchemy ORM

In [3]:
# Set up query engine. 'echo=True is the default - will keep a log of activities'

engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [4]:
# Reflect an existing database into a new model
Base = automap_base()
# Reflect the tables
Base.prepare(engine, reflect=True)

In [5]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [6]:
# Another way to get table names from SQL-lite
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

## Exploratory Climate Analysis

In [9]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [10]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [11]:
# Display details of 'measurement' table
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c['type'])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [12]:
# DISPLY number of line items measurement, and remove tuple form
result, = engine.execute('SELECT COUNT(*) FROM measurement').fetchall()[0]
print(result,)

19550


In [13]:
# Display details of 'station' table
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c['type'])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [14]:
# DISPLY number of line items station, and remove tuple form

result, = engine.execute('SELECT COUNT(*) FROM station').fetchall()[0]
print(result,)

9


In [15]:
# FULL INNTER JOIN BOTH THE MEASUREMENT AND STATION TABLE

# engine.execute('SELECT measurement.*, station.name, station.latitude FROM measurement INNER JOIN station ON measurement.station = station.station;').fetchall()
join_result = engine.execute('SELECT * FROM measurement INNER JOIN station ON measurement.station = station.station;').fetchall()
join_result

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (6, 'USC00519397', '2010-01-07', 0.06, 70.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (7, 'USC00519397', '2010-01-08', 0.0, 64.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (8, 'USC00519397', '2010-01-09', 0.0, 68.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (9, 'USC00519397', '2010-01-10', 0.0, 73.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.271

In [16]:
# Another way to PERFORM AN INNER JOIN ON THE MEASUREMENT AND STATION TABLES

engine.execute('SELECT measurement.*, station.* FROM measurement, station WHERE measurement.station=station.station;').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (6, 'USC00519397', '2010-01-07', 0.06, 70.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (7, 'USC00519397', '2010-01-08', 0.0, 64.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (8, 'USC00519397', '2010-01-09', 0.0, 68.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (9, 'USC00519397', '2010-01-10', 0.0, 73.0, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.271

In [17]:
# Query last date of the measurement file

last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]

print(last_date) 

last_date_measurement = dt.date(2017, 8 ,23)

2017-08-23


In [18]:
# Calculate the date 1 year delta of the "last date measurement"

one_year_ago = last_date_measurement - dt.timedelta(days=365)
print(one_year_ago)

2016-08-23
